# Lab 7-1: Markov Chains - Weather, as in class lecture
(Steven Pestana, 2019. Derived from CEE599_MarkovChain_Lab.m, Jessica Lundquist, Oct 23, 2015)

(see Markov Chain Weather Example in Lecture notes, except note that we now have an initial condition, which would be like just saying today's weather was dry) 

In [1]:
import numpy as np
from scipy.linalg import eig
import scipy.stats as stats
from scipy import sparse
import matplotlib.pyplot as plt

%matplotlib inline

****

## Example - Dry vs Rain

We are going to use a Markov Chain to look at the conditional probabilities of a dry day vs a wet day, at steps of 1 day.

Let's define two states, in our model someone can either live in the city or the suburbs:
 - dry = 0
 - rain = 1

Set up the Markov probability matrix, start with just an empty 2x2 array:

In [2]:
Pmarkov = np.zeros([2,2]) # create an empty 2-by-2 numpy array
Pmarkov

array([[0., 0.],
       [0., 0.]])

Given toay is dry, we have 80% chance of being dry tomorrow and 20% chance of rain:

In [5]:
# Assign our given values to the table:
Pmarkov[0,0] = 0.8 # note that we use the array indices to describe probability of going from state 0 to state 0
Pmarkov[0,1] = 0.2 # probability of going from state 0 to state 1

Given today has rain, we have a 60% chance of dry tomorrow and a 40% chance of rain:  (not really Seattle in November)

In [6]:
# Assign our given values to the table:
Pmarkov[1,0] = 0.60 # probability of going from state 1 to state 0
Pmarkov[1,1] = 0.40 # probability of going from state 1 to state 1

View the complete matrix:

In [7]:
print(Pmarkov)

[[0.8 0.2]
 [0.6 0.4]]


Before going into matrix notation, let's think this through. In the initial state, today is dry.

**Step 1st day** using algebra

Take one step through the Markov chain. What is the probability of dry tomorrow, given dry today?

$p_{dry,1}$ = 0.8 * 1 + 0.2 * 0

In [8]:
pdry0=1
pwet0=0
pdry1 = Pmarkov[0,0]*pdry0 + Pmarkov[1,0]*pwet0
print(pdry1)

0.8


And what is the chance that tomorrow is wet?

$p_{wet,1}$ = 0.2 * 1 + 0.4 * 0

In [9]:
pwet1 = Pmarkov[0,1]*pdry0 + Pmarkov[1,1]*pwet0
print(pwet1)

0.2


Draw a picture on scratch paper of what you are doing in terms of matrix multiplication.

**Step 1st day** using matrix multiplication

Now use matrix algebra using numpy to get the same answer and check your work.

In [10]:
# initial state of populations for state 0 and state 1
percent0 = np.array([pdry0, pwet0])

# multiply initial states with Markov matrix
percent1 = np.dot(percent0, Pmarkov)

# check values of x1 here against what you got for pcity1 and pusuburb1
print(percent1)

[0.8 0.2]


**Step 2nd day**

Now we can make this the current state and take a second step with the matrix (using both one step and matrix notation to check our work):

In [11]:
# take a second step with the Markov matrix:
pdry2 = Pmarkov[0,0]*pdry1 + Pmarkov[1,0]*pwet1
pwet2=Pmarkov[0,1]*pdry1 + Pmarkov[1,1]*pwet1
print(pdry2,pwet2)

# take a second step with the Markov matrix (using np.dot)
percent2 = np.dot(percent1,Pmarkov)
print(percent2[0],percent2[1])


0.7600000000000001 0.24000000000000005
0.7600000000000001 0.24000000000000005


Good, we got the same answer (within rounding error), let's just use matrix math for the rest of the lab.

**Step 3rd year**

We can also make multiple steps by raising the Markov matrix to a power:

In [12]:
# three steps all at once by taking our Pmarkov matrix to the 3rd power, then multipying by our original population percentage vector
percent3 = np.dot(percent0, np.linalg.matrix_power(Pmarkov,3))
print(percent3)

# we can also take a 3rd step by just multiplying through from the 2nd year
percent3_check = np.dot(percent2,Pmarkov)
print(percent3_check)

[0.752 0.248]
[0.752 0.248]


In [13]:
# 10 steps, then 100 steps:
percent10 = np.dot(percent0, np.linalg.matrix_power(Pmarkov,10))
percent100 = np.dot(percent0, np.linalg.matrix_power(Pmarkov,100))
print(percent10)
print(percent100)

[0.75000003 0.24999997]
[0.75 0.25]


**Find the steady state probabilities** using the eigenvector function

And now, use the eigenvector function in numpy to see if x100 is the same as the steady state matrix:

Here, we are using the concept that for a square matrix A, the column vector v is an eigenvector of A is theere is a scalar multiple $\lambda$, such that
multipliplying it times v is the same as applying the matrix multiplication of A on v.

$ \lambda v = Av $

This fits the steady state matrix definition of

$ \pi = \pi P $

for the case of $\lambda$ = 1 and P = the transpose of A.


In [23]:
# So, we solve the eigenvalue problem of the transpose of the markov matrix A.
# See https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eig.html#scipy.linalg.eig 
# Here, W are the eigenvalues and V are the eigenvectors

W, V = eig(Pmarkov.T)
print(Pmarkov.T)

[[0.8 0.6]
 [0.2 0.4]]


In [15]:
V

array([[ 0.9486833 , -0.70710678],
       [ 0.31622777,  0.70710678]])

Find the column of V associated with the eigenvalue of 1, but allow for some margin of round-off error

(from the function documentation: the column ```V[:,i]``` is the eigenvector corresponding to the eigenvalue ```W[i]```)

In [16]:
# First, we can inspect W by eye (since this is a small matrix)
W

array([1. +0.j, 0.2+0.j])

We want the firt column (index 0)

In [17]:
V[:,0]

array([0.9486833 , 0.31622777])

But these need to be normalized first

In [19]:
p = V[:,0] / np.sum(V[:,0])
p

array([0.75, 0.25])

Which matches what we expect from our prior calculations. Yay!

We can write a function to find this for us:

In [21]:
def eig_pi(X, err=1e-9):
    '''Find the value of W closest to 1 (within some round-off error) and then look at the associated column of V.
    Input: tuple of (W (eigenvalue matrix), V (eigenvectors)) that are output from scipy.linalg.eig()'''
    [W, V] = X
    # find the column where the eigenvalue, W, is within 1 +/- err
    col1 = np.where((W>=1-err) & (W<=1+err))
    # normalize the eigenvector, V, for that column, so that it sums to one
    p = V[:,col1] / np.sum(V[:,col1]) 
    return p

In [22]:
p = eig_pi(eig(Pmarkov.T))

print('Eigenvector Steady State Probabilities: {}'.format(p.T[0][0]))

# Yes, these are very close to the values we got for percent100
print('Markov Chain 100-step Probabilities: {}'.format(percent100))

Eigenvector Steady State Probabilities: [0.75 0.25]
Markov Chain 100-step Probabilities: [0.75 0.25]


***

## Example - Create a Markov Matrix from data

Now given a timeseries of four states, determine the Markov Matrix and use it to generate a timeseries

In [29]:
# first, read in data that I generated (sequence of states 1, 2, 3, and 4):
data = np.genfromtxt('../data/markov_random4.txt',dtype=int)

print(data)

[1 3 3 3 4 1 4 1 2 1 2 1 1 2 4 3 2 4 4 4 1 2 4 1 2 1 2 1 2 2 1 2 2 2 4 3 1
 1 4 1 1 2 4 1 3 3 3 3 2 1 2 1 3 2 2 4 2 2 4 3 1 4 4 3 2 2 4 2 4 3 2 3 4 1
 2 4 1 1 3 3 4 1 2 3 2 4 1 1 4 1 1 3 3 1 2 2 2 1 2 4 1 3 4 3 3 2 2 4 1 2 2
 4 1 2 1 4 1 1 2 3 2 4 1 4 3 4 3 1 2 2 2 4 3 2 2 3 3 1 2 1 2 4 1 4 3 3 4 3
 4 2 4 3 3 3 3 2 3 2 4 1 4 1 4 3 3 4 3 3 3 4 3 3 3 2 1 2 4 1 4 1 3 3 2 1 2
 1 2 2 4 3 4 2 4 1 3 2 4 3 4 3 4 3 4 3 3 3 4 1 3 2 4 3 3 3 3 1 1 4 4 1 3 2
 2 4 3 3 4 4 2 1 2 3 3 4 1 1 2 1 4 2 4 4 1 2 4 3 1 2 4 4 3 2 4 1 4 1 2 3 4
 1 1 2 2 1 2 2 4 3 3 3 4 1 2 1 2 4 3 4 1 4 1 4 2 4 1 3 2 2 2 4 1 2 1 4 1 3
 2 2 4 1 2 4 1 2 1 3 3 1 2 2 4 3 3 3 2 4 3 2 4 1 2 2 2 4 1 1 4 1 3 1 2 3 3
 3 3 1 2 4 1 2 2 1 2 4 1 1 3 2 4 4 3 3 3 3 3 4 1 4 3 4 2 1 4 2 2 3 2 4 1 4
 1 4 3 1 3 4 3 4 1 2 2 1 1 4 1 3 2 2 4 3 4 4 3 2 2 4 1 2 2 1 3 2 4 1 4 3 3
 3 2 1 2 3 2 4 1 3 3 2 2 1 2 4 1 1 4 3 4 4 4 4 1 2 3 3 2 2 1 4 3 3 3 3 3 4
 1 3 2 4 3 2 3 1 1 2 4 2 2 4 3 4 1 4 1 3 4 3 3 3 2 2 4 1 2 1 2 1 1 3 1 2 2
 4 2 1 4 1 2 1 2 4 4 2 1 

In [37]:
# This counts the transitions from each state to the next and marks that count
testdata= (np.ones_like(data[:-1]), (data[:-1], data[1:]))
#print(testdata)
S = sparse.csr_matrix((np.ones_like(data[:-1]), (data[:-1], data[1:])), dtype=float)
print(S)
# This converts those counts to matrix form
tm = S.todense()
print(tm)

  (1, 1)	16.0
  (1, 2)	53.0
  (1, 3)	22.0
  (1, 4)	28.0
  (2, 1)	31.0
  (2, 2)	34.0
  (2, 3)	12.0
  (2, 4)	56.0
  (3, 1)	13.0
  (3, 2)	34.0
  (3, 3)	48.0
  (3, 4)	28.0
  (4, 1)	58.0
  (4, 2)	12.0
  (4, 3)	41.0
  (4, 4)	13.0
[[ 0.  0.  0.  0.  0.]
 [ 0. 16. 53. 22. 28.]
 [ 0. 31. 34. 12. 56.]
 [ 0. 13. 34. 48. 28.]
 [ 0. 58. 12. 41. 13.]]


**Take a look at the matrix above, what does this represent?**

We had 16 counts when the series transitioned from 1 to 1, 53 counts when it transitioned from 1 to 2, etc.
We want to transition these from counts to frequencies.
To do this, we need to normalize the transition matrix to get probabilities

In [22]:
tm_norm = tm / tm.sum(axis=1)
    
print(tm_norm) # This is our normalized transition matrix.

[[       nan        nan        nan        nan        nan]
 [0.         0.13445378 0.44537815 0.18487395 0.23529412]
 [0.         0.23308271 0.2556391  0.09022556 0.42105263]
 [0.         0.10569106 0.27642276 0.3902439  0.22764228]
 [0.         0.46774194 0.09677419 0.33064516 0.10483871]]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [23]:
# We take the above probabilities of transitions, and turn them into discrete CDF's.
# These will allow us to map random numbers generated from a uniform distribution into 
# transitions that follow these probability rules.
tm_cdf = np.cumsum(tm_norm,1)

Now, we want a "random walk" for 2000 years that follows these transition probabilities.

In [24]:
n_years = 2000
q = np.random.uniform(0,1,n_years); # uniformly distributed random numbers n_years long

initialstate = 2; # give it an initial state, doesn't really matter which

Nrand = np.zeros_like(q) # initialize an array of the proper size, with the initial state
Nrand[0] = initialstate;

# Now, just like we did when we created monte carlo simulations from empirical CDFs,
# we use our uniform random numbers to look up the next state in the transition matrix
for i in range(1,n_years):
    if q[i] <= tm_cdf[int(Nrand[i-1]),1]: #probability of transitioning from state i to 1
        Nrand[i] = 1;
    elif q[i] <= tm_cdf[int(Nrand[i-1]),2]: #transition from state i to 2
        Nrand[i] = 2;
    elif q[i] <= tm_cdf[int(Nrand[i-1]),3]: #transition from state i to 3
        Nrand[i] = 3;
    else:
        Nrand[i] = 4;

In [25]:
# And how many times did state 3 appear 4 times?
Test3 = [Nrand[0:-3], Nrand[1:-2], Nrand[2:-1], Nrand[3:]] # stack our data 4 times, shifting it to the right by 1 each time
Test3 = np.stack(Test3, axis=1)

G2 = np.where((np.max(Test3, axis=1) == 3) & (np.min(Test3, axis=1) == 3))
# if both the maximum and the minimum are 3, then we have 4 threes in our sequence

frequencyof4threes = G2[0].size / Test3.shape[0]

print('Frequency of four 3s in a row = {}%'.format(100*np.round(frequencyof4threes,3)))

Frequency of four 3s in a row = 1.3%
